In [1]:
import pyaudio
import numpy as np
import keyboard
import time

RATE = 44100                                       # サンプリング周波数
CHUNK = 1024                                       # バッファサイズ
CHANNEL_IN = 2                                     # ステレオ入力とする
CHANNEL_OUT = 2                                    # ステレオ出力とする
prc= 0                                             # 素通しか処理音の切り替えフラグ

M  = CHUNK                                         # 入力信号の記憶数
xL = np.zeros(M)                                   # 左チャネル入力信号
xR = np.zeros(M)                                   # 右チャネル入力信号
xL1= np.zeros(M*2)                                 # 左チャネルの遅延入力信号
xR1= np.zeros(M*2)                                 # 右チャネルの遅延入力信号
yL = np.zeros(M)                                   # 左チャネル出力信号
yR = np.zeros(M)                                   # 右チャネル出力信号
x1 = np.zeros(M*2)                                 # 左チャネルの入力信号
x2 = np.zeros(M*2)                                 # 右チャネルの入力信号
y1 = np.zeros(M*3)                                 # 左チャネルの入力信号
y2 = np.zeros(M*3)                                 # 右チャネルの入力信号
w  = np.hamming(M*2)                               # 窓関数
n  = 0                                             # 時間インデクス

DL = np.zeros(M*2)                                 # 左チャネルのノイズ推定値
DR = np.zeros(M*2)                                 # 右チャネルのノイズ推定値
q  = 1                                             # ノイズ推定フレームのカウンタ

def signal_proc(input_buff, dtype=np.int16):
    input_data = np.fromstring(input_buff, dtype=dtype) # バッファから入力信号を読み込む
    input_data = input_data.astype(np.float32)          # 16ビット整数 --> 32ビット浮動小数点
    
    global xL, xR, n, M                            # グローバル変数
    global xL1, xR1, y1, y2, y, w                  # グローバル変数
    global q, DL, DR                               # グローバル変数
    #######################
    ##  パラメータ設定   ##
    #######################
    CFH = 10000                                      # 高域側カットオフ周波数[Hz]
    kch = int( CFH/(RATE/(CHUNK*2)) )                # 高域側カットオフ周波数番号
    CFL = 300                                      # 低域側カットオフ周波数[Hz]
    kcl = int( CFL/(RATE/(CHUNK*2)) )                # 低域側カットオフ周波数番号
 
    Th = 0.035;                                 # ノイズゲートのしきい値

    p=1.4;                                         # 振幅スペクトルのべき
    Q  = 10                                        # ノイズ推定フレームの数
    s=1.75                                            #減算定数

    y  = np.zeros(CHUNK*2)                         # ステレオ出力信号 (配列の長さはバッファサイズの2倍)
    xL = input_data[0::2]/32768                    # 左チャネル入力信号
    xR = input_data[1::2]/32768                    # 右チャネル入力信号

    xL1[M::] = xL;                                 # XL1に左チャネル入力を追加
    xR1[M::] = xR;                                 # XR1に右チャネル入力を追加

    x1  = xL1 * w                                  # 左チャネル入力に窓を掛ける
    x2  = xR1 * w                                  # 右チャネル入力に窓を掛ける
    XL  = np.fft.rfft(x1)                          # 左チャネル信号をFFT
    XR  = np.fft.rfft(x2)                          # 右チャネル信号をFFT

    if prc==0:
        YL  = XL                                   # 左チャネル出力信号 (未処理)
        YR  = XR                                   # 右チャネル出力信号 (未処理)
    if prc==1:
    ############################################
    ##                                        ##
    ##   現在時刻 n における信号処理          ##
    ##                                        ##
    ##   入力信号のFFT XL と XR を処理して    ##
    ##   出力信号のFFT YL と YR をつくる      ##
    ##                                        ##
    ############################################

        XampL = np.abs(XL)                         # 入力の振幅スペクトル
        XphsL = np.angle(XL)                       # 入力の位相スペクトル
        z0L   = np.fft.irfft( np.power(XampL,p) )  # ゼロ位相変換
        z0L   = z0L-s*DL                             # 減算処理
        YampL = np.fft.rfft (z0L)                  # 逆ゼロ位相変換
        YampL = np.power(YampL, 1/p)               # 逆ゼロ位相変換
        YphsL = XphsL                              # 出力の位相スペクトル
        YL    = YampL * np.exp( 1j*YphsL )         # 出力の複素スペクトル
        YL[kch::]=0                                 # kch番目の周波数より高域をカット
        YL[0:kcl]=0                                 # kcl番目の周波数より低域をカット
        if  xL[n] < Th and xL[n] > -Th:    # しきい値の範囲内の振幅なら0にする
                YL[n] = 0
        
        
        XampR = np.abs(XR)                         # 入力の振幅スペクトル
        XphsR = np.angle(XR)                       # 入力の位相スペクトル
        z0R   = np.fft.irfft( np.power(XampR,p) )  # ゼロ位相変換
        z0R   = z0R-s*DR                             # 減算処理
        YampR = np.fft.rfft (z0R)                  # 逆ゼロ位相変換
        YampR = np.power(YampR, 1/p)               # 逆ゼロ位相変換
        YphsR = XphsR                              # 出力の位相スペクトル
        YR    = YampR * np.exp( 1j*YphsR )         # 出力の複素スペクトル
        YR[kch::]=0                                 # kch番目の周波数より高域をカット
        YR[0:kcl]=0                                # kcl番目の周波数より低域をカット
        if  xR[n] < Th and xR[n] > -Th:    # しきい値の範囲内の振幅なら0にする
                YR[n] = 0

        if q <= Q:                                 # q<=Qならばノイズ推定
            DL = DL + z0L/Q                        # Qフレームのゼロ位相信号をノイズにする
            DR = DR + z0R/Q                        # Qフレームのゼロ位相信号をノイズにする
            q  = q  + 1                            # qは[space]を押すたびに初期化される

    
    ########################################
    ##            ここまで                ##
    ########################################
    y1[1*M::]  =  y1[1*M::] + np.real( np.fft.irfft(YL) )    # IFFTしてオーバラップ加算（左）
    y2[1*M::]  =  y2[1*M::] + np.real( np.fft.irfft(YR) )    # IFFTしてオーバラップ加算（右）
        
    y1[0:2*M] = y1[M::]                            # 左チャネル出力信号シフト
    y1[2*M::] = 0                                  # 後半は0に初期化
    y2[0:2*M] = y2[M::]                            # 右チャネル出力信号シフト
    y2[2*M::] = 0                                  # 後半は0に初期化
    
    yL  =  y1[0:M]                                 # 左の出力信号をバッファサイズだけ記録
    yR  =  y2[0:M]                                 # 右の出力信号をバッファサイズだけ記録

    y[0::2]=yL*32767                               # 左チャネル出力信号をyに格納
    y[1::2]=yR*32767                               # 右チャネル出力信号をyに格納
    
    xL1[0:M] = xL                                  # 左チャネル入力信号シフト
    xR1[0:M] = xR                                  # 右チャネル入力信号シフト

    # Convert nd-array into framebuffer
    output_buff = y.astype(dtype).tostring()       # yを出力バッファに送る
    return output_buff

p = pyaudio.PyAudio()                              # オーディオデバイスの情報取得

stream_in = p.open(                                # 入力（マイク）デバイスの設定
        format=pyaudio.paInt16,
        channels=CHANNEL_IN,
        rate = RATE,
        frames_per_buffer=CHUNK,
        input = True,
        output = False,
    )

stream_out = p.open(                               # 出力（スピーカ）デバイスの設定
        format=pyaudio.paInt16,
        channels=CHANNEL_OUT,
        rate=RATE,
        frames_per_buffer=CHUNK,
        input=False,
        output=True,
    )

########################################
#          使い方の説明表示            #
########################################
print("[space]--> Through <=> Processing")
print("[Enter]--> End")
print("Through")

################################################
#  マイク入力 - スピーカ出力の無限ループを回す #
################################################
while stream_in.is_active() and stream_out.is_active():
    input_buff  = stream_in.read(CHUNK)
    output_buff = signal_proc(input_buff)
    stream_out.write(output_buff)
    ###########################################
    ##      素通しと処理音を切り替える       ##
    ###########################################
   
    if True:                                               # 何のキーか取得
        if keyboard.is_pressed("space"):                   # [Space]キーなら，素通しと処理音を切り替える
            prc = -prc+1                # 合わせてフラグを切り替え
            if prc==1:                  # 現在の状態を表示
                print("Processing")
                time.sleep(0.1)
            if prc==0:
                print("Through")
                q = 1; DL=0*DL; DR=0*DR # パラメータの初期化
                time.sleep(0.1)
        if keyboard.is_pressed("enter"):
            break  # [Enter]キーなら終了
                
    
stream_in.stop_stream()
stream_in.close()
stream_out.stop_stream()
stream_out.close()
p.terminate()

[space]--> Through <=> Processing
[Enter]--> End
Through


C:\Users\orang\AppData\Local\Temp\ipykernel_4120\4031414178.py:31: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  input_data = np.fromstring(input_buff, dtype=dtype) # バッファから入力信号を読み込む
C:\Users\orang\AppData\Local\Temp\ipykernel_4120\4031414178.py:130: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  output_buff = y.astype(dtype).tostring()       # yを出力バッファに送る


Processing
Through
Processing
